In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymysql

In [3]:
# Functions

# reverse the data of argument
def reverseData(data):
    data = data[::-1]
    return data

def getRound(data, decimals = 3):
    return np.around(data, decimals)

## given two curve a, b, then return the inversection point of two curves
def FindTheBuyPoint(a, b):
    buyPoint = np.argwhere(np.diff(np.sign(a - b)) > 0).reshape(-1) + 0
    return buyPoint

def FindTheSellPoint(a, b):
    sellPoint = np.argwhere(np.diff(np.sign(a - b)) < 0).reshape(-1) + 0
    return sellPoint

def GetDataFromDB(ip, userName, psw, tableName, sql):
    try:
        db = pymysql.connect(ip, userName, psw, tableName)
    except:
        print ("Error: unable to connect to DB")
        
    #create a cursor
    cursor = db.cursor()
    results = []
    try:
        cursor.execute(sql)
        results = cursor.fetchall()
    except:
        print ("Error: unable to fetch data from DB")
    
    return results
    db.close()

def SMA(data, period):
    if (type(data) == pd.core.series.Series):
        return data.rolling(window = period).mean()
    else:
        print ("Error: Wrong input, SMA(pandas.core.series.Series, integer)")
        
def EMA(data, period):
    if (type(data) == pd.core.series.Series):
        return data.ewm(span = period, adjust = False).mean()
    else:
        print ("Error: Wrong input, EMA(pandas.core.series.Series, integer)")
        
def RSI(data, period, Uprevious = None, Dprevious = None):
    if (type(data) == pd.core.series.Series):
        delta = data.diff()
        up, down = delta.copy(), delta.copy()
        up[up < 0] = 0
        down[down > 0] = 0
        down = abs(down)
        Up = SMA(up, period)
        Up = Up.fillna(0)
        if(Uprevious == None):
            for i in range(period,len(Up)):
                Up[i] = getRound((Up[i - 1] * (period - 1) + up[i]) / period)
        else:
            Up[period - 1] = getRound((Up[i - 1] * (period - 1) + up[i]) / period)
            
        Down = SMA(down, period)
        Down = Down.fillna(0)
        if(Uprevious == None):
            for i in range(period,len(Down)):
                Down[i] = getRound((Down[i - 1] * (period - 1) + down[i]) / period)
        else:
            Down[period - 1] = getRound((Dprevious * (period - 1) + down[i]) / period)

        rsi = 100 * (Up / (Up + Down))
        rsi = rsi.fillna(0)
        return getRound(rsi, 2), Up, Down
    else:
        print ("Error: Wrong input, RSI(pandas.core.series.Series, integer)")
        
def MACD(data, period = []):
    if (type(data) == pd.core.frame.DataFrame and len(period) == 3):   
        di = (data['High'] + data['Low'] + 2.0 * data['Close']) / 4.0
        ema12 = SMA(di, period[0])
        ema12 = ema12.fillna(0)
        for i in range(period[0] + 1, len(ema12)):
            ema12[i] = (ema12[i - 1] * (period[0] - 1) + di[i] * 2.0) / (period[0] + 1)
    
        ema26 = SMA(di, period[1])
        ema26 = ema26.fillna(0)
        for i in range(period[1] + 1, len(ema26)):
            ema26[i] = (ema26[i - 1] * (period[1] - 1) + di[i] * 2.0) / (period[1] + 1)
    
        dif = ema12 - ema26

        dem = SMA(dif, period[2])
        dem = dem.fillna(0)
        for i in range(period[2] + 1, len(dem)):
            dem[i] = (dem[i - 1] * (period[2] - 1) + dif[i] * 2.0) / (period[2] + 1)
        return {'dif':dif, 'dem':dem}
    else:
        if(type(data) == pd.core.frame.DataFrame):
            print ("Error: Wrong input, MACD(pandas.core.frame.DataFrame, list of integer)")
        elif(len(period) == 3):
            print ("Error: number of content in list do not equal to 3")
            
def KD(data, result = {}):
    close = data['Close'].copy()
    for i in range(0,9):
        close[i] = 0

    data['RSV'] = (( data['Close'] - data['Low'].rolling(window = 9).min()) / (data['High'].rolling(window = 9).max() - data['Low'].rolling(window = 9).min()))
    data['RSV'] = data['RSV'].fillna(0)
    if(not result):
        result = {
            'K9':[0],
            'D9' :[0]
        }
    #calculate everyday's KD
    for i in range(1, len(data.index)):
        K9_value = (1.0/3.0) * data['RSV'][i] + (2.0 / 3.0) * result['K9'][i - 1]
        result['K9'].append(getRound(K9_value, 5))
        D9_value = (2.0/3.0) * result['D9'][i - 1] + (1.0 / 3.0) * result['K9'][i]
        result['D9'].append(getRound(D9_value, 5))
        
    return result

In [4]:
# Real Time Function for value of Indicators

# RSI
def rsiR(date):
    # pull the last 9 data needed
    period = 9
    sql = "SELECT DATE,LAST FROM btc_usd ORDER BY DATE DESC LIMIT %s" % (str(period))
    result = GetDataFromDB("140.118.126.136", "123", "1234567890", "test", sql)
    
    # organized the data to 2 list
    Dates = []
    ClosePrices = []

    for i in range(len(result)):
        Dates.append(result[i][0])
        ClosePrices.append(result[i][1])
        
    # reorganized the data by reverse order
    Dates = reverseData(Dates)
    ClosePrices = reverseData(ClosePrices)
    
    # reorganized the data to dataframe type
    CoinPriceData = {
        'Date' : Dates,
        'Close' : ClosePrices
    }
    CoinPriceData = pd.DataFrame(CoinPriceData)
    
    # calculate the value of rsi
    period = period - 1
    resultOfrsiR, _, _ = RSI(CoinPriceData['Close'], period);
    targetRSI = getRound(float(resultOfrsiR[len(resultOfrsiR) - 1]))

    # push the result to database
    sql = "UPDATE INDICATORS SET RSI = %s WHERE DATE = %s" % (targetRSI, date)
    print(sql)
    #result = GetDataFromDB("140.118.126.136", "123", "1234567890", "test", sql)
    
# SMA
def smaR(Date):
    # pull the last 100 data needed
    period = 240
    sql = "SELECT DATE,LAST FROM btc_usd ORDER BY DATE DESC LIMIT %s" % (str(period))
    result240 = GetDataFromDB("140.118.126.136", "123", "1234567890", "test", sql)
    
    # organized the data to 2 list
    Dates = []
    ClosePrices = []
    for i in range(len(result100)):
        Dates.append(result240[i][0])
        ClosePrices.append(result240[i][1])
    
    # reorganized the data by reverse order
    Dates = reverseData(Dates)
    ClosePrices = reverseData(ClosePrices)

    # reorganized the data to dataframe type
    CoinPriceData = {
        'Date' : Dates,
        'Close' : ClosePrices
    }
    CoinPriceData = pd.DataFrame(CoinPriceData)
    
    # calculate the value of rsi
    period = period
    resultOfsma240R = SMA(CoinPriceData['Close'], period);
    targetSMA240 = getRound(float(resultOfsma240R[len(resultOfsma240R) - 1]))
    
    period = 120
    resultOfsma120R = CoinPriceData['Close'].tail(120).reset_index()
    del resultOfsma120R['index']
    resultOfsma120R = SMA(resultOfsma120R['Close'], period);
    targetSMA120 = getRound(float(resultOfsma120R[len(resultOfsma120R) - 1]))
    
    period = 60
    resultOfsma60R = CoinPriceData['Close'].tail(60).reset_index()
    del resultOfsma60R['index']
    resultOfsma60R = SMA(resultOfsma60R['Close'], period);
    targetSMA60 = getRound(float(resultOfsma60R[len(resultOfsma60R) - 1]))
    
    period = 20
    resultOfsma20R = CoinPriceData['Close'].tail(20).reset_index()
    del resultOfsma20R['index']
    resultOfsma20R = SMA(resultOfsma20R['Close'], period);
    targetSMA20 = getRound(float(resultOfsma20R[len(resultOfsma20R) - 1]))
    
    period = 10
    resultOfsma10R = CoinPriceData['Close'].tail(10).reset_index()
    del resultOfsma10R['index']
    resultOfsma10R = SMA(resultOfsma10R['Close'], period);
    targetSMA10 = getRound(float(resultOfsma10R[len(resultOfsma10R) - 1]))
    
    period = 5
    resultOfsma5R = CoinPriceData['Close'].tail(5).reset_index()
    del resultOfsma5R['index']
    resultOfsma5R = SMA(resultOfsma5R['Close'], period);
    targetSMA5 = getRound(float(resultOfsma5R[len(resultOfsma5R) - 1]))
    
    # push the result to database
    sql = "UPDATE INDICATORS SET SMA240 = %s, SMA120 = %s, SMA60 = %s, SMA20 = %s, SMA10 = %s, SMA5 = %s WHERE DATE = %s" % (targetSMA240, targetSMA120, targetSMA60, targetSMA20, targetSMA120, targetSMA5, Date)
    print(sql)
    #result = GetDataFromDB("140.118.126.136", "123", "1234567890", "test", sql)
    
# KD
def kdR(date):
    # pull the last 2 data needed
    period = 2
    sql = "SELECT DATE,LAST, HIGH, LOW FROM btc_usd ORDER BY DATE DESC LIMIT %s" % (str(period))
    result = GetDataFromDB("140.118.126.136", "123", "1234567890", "test", sql)
    sql = "SELECT K, D FROM indicators ORDER BY DATE DESC LIMIT %s" % (str(period))
    kdresult = GetDataFromDB("140.118.126.136", "123", "1234567890", "test", sql)
    
    # organized the data to 2 list
    Dates = []
    ClosePrices = []
    K = []
    D = []
    for i in range(len(result)):
        Dates.append(result[i][0])
        ClosePrices.append(result[i][1])
        K.append(kdresult[i][0])
        D.append(kdresult[i][1])
    
    # reorganized the data by reverse order
    Dates = reverseData(Dates)
    ClosePrices = reverseData(ClosePrices)
    K = reverseData(K)
    D = reverseData(D)
    
    # reorganized the data to dataframe type
    CoinPriceData = {
        'Date' : Dates,
        'Close' : ClosePrices
    }
    kd = {
        'K9' : K,
        'D9' : D
    }
    CoinPriceData = pd.DataFrame(CoinPriceData)
    kd = pd.DataFrame(kd)
    
    # calculate the value of rsi
    resultOfkdR = KD(CoinPriceData, kd);
    targetK = getRound(float(resultOfkdR['K9'][len(resultOfkdR) - 1]))
    targetD = getRound(float(resultOfkdR['D9'][len(resultOfkdR) - 1]))
    
    # push the result to database
    sql = "UPDATE INDICATORS SET K = %s , D = %s WHERE DATE = %s" % (targetK, targetD, date)
    print(sql)
    #result = GetDataFromDB("140.118.126.136", "123", "1234567890", "test", sql)
    
# MACD
def macdR(date):
    # pull the last 2 data needed
    # 12 26 9
    period = 26
    sql = "SELECT DATE,LAST, HIGH, LOW FROM btc_usd ORDER BY DATE DESC LIMIT %s" % (str(period))
    result = GetDataFromDB("140.118.126.136", "123", "1234567890", "test", sql)
    
    # organized the data to 2 list
    Dates = []
    ClosePrices = []
    HighPrices = []
    LowPrices = []
    for i in range(len(result)):
        Dates.append(result[i][0])
        ClosePrices.append(result[i][1])
        HighPrices.append(result[i][2])
        LowPrices.append(result[i][3])
    
    # reorganized the data to dataframe type
    CoinPriceData = {
        'Date' : Dates,
        'Close' : ClosePrices,
        'High' : HighPrices,
        'Low' : LowPrices
    }
    CoinPriceData = pd.DataFrame(CoinPriceData)
    
    # calculate the value of rsi
    resultOfmacdR = MACD(CoinPriceData, [12, 26, 9]);
    targetDIF = getRound(float(resultOfkdR['dif'][len(resultOfkdR) - 1]))
    targetDEM = getRound(float(resultOfkdR['dem'][len(resultOfkdR) - 1]))
    
    # push the result to database
    sql = "UPDATE INDICATORS SET dif = %s , dem = %s WHERE DATE = %s" % (targetDIF, targetDEM, date)
    print(sql)
    #result = GetDataFromDB("140.118.126.136", "123", "1234567890", "test", sql)

In [5]:
# define a class - indicators, which contains all indicators that calculate by the data
class Indicators:
    data = pd.DataFrame()
    EMA = pd.DataFrame()
    SMA = pd.DataFrame()
    RSI = pd.DataFrame()
    KD = pd.DataFrame()
    MACD = pd.DataFrame()
    
    def __init__(self, inputData):
        self.data = inputData
        self.SMA = self.setSMA([5, 10, 20, 60, 120, 240])
        self.EMA = self.setEMA([5, 20, 100])
        self.RSI = self.setRSI(14)
        self.KD = self.setKD()
        self.MACD = self.setMACD([12, 26, 9])
    
    def setSMA(self, periods = []):
        smaResults = []
        for i in range(len(periods)):
            if (type(periods[i]) != int):
                print ("Error: inputData is not a list of integer")
                return
            else:
                smaResults.append(SMA(self.data['Close'], periods[i]))
                
        return pd.DataFrame(dict((("SMA" + str(periods[i])), smaResults[i]) for i in range(0, len(smaResults))))
    
    def getSMA(self):
        return self.SMA
    
    def setEMA(self, periods = []):
        emaResults = []
        for i in range(len(periods)):
            if (type(periods[i]) != int):
                print ("Error: inputData is not a list of integer")
                return
            else:
                emaResults.append(EMA(self.data['Close'], periods[i]))
                
        return pd.DataFrame(dict((("EMA" + str(periods[i])), emaResults[i]) for i in range(0, len(emaResults))))
    
    def getEMA(self):
        return self.EMA
    
    def setRSI(self, period):
        if (type(period) != int):
            print ("Error: inputData is not an integer")
            return
        
        rsiResults = RSI(self.data['Close'], period)
        return pd.DataFrame({"RSI": list(rsiResults)})
    
    def getRSI(self):
        return self.RSI
    
    def setKD(self):
        kdResults = KD(self.data)
        return pd.DataFrame(kdResults)
    
    def getKD(self):
        return self.KD
    
    def setMACD(self, periods = []):
        macdResults = []
        for i in range(len(periods)):
            if (type(periods[i]) != int):
                print ("Error: inputData is not a list of integer")
                return
        macdResults = MACD(self.data, periods)
        return pd.DataFrame(macdResults)
    
    def getMACD(self):
        return self.MACD
    
    def getLong_ShortPoints(self, indicator):
        if (type(indicator) != str):
            print ("Error: the type of argument is wrong : it should be str type")
            return
        longPoints = []
        shortPoints = []
        if(indicator.lower() == 'sma'):
            longPoints_5_20 = FindTheBuyPoint(self.SMA['SMA5'].head(365), self.SMA['SMA20'].head(365))
            longPoints_5_100 = FindTheBuyPoint(self.SMA['SMA5'].head(365), self.SMA['SMA100'].head(365))
            longPoints_20_100 = FindTheBuyPoint(self.SMA['SMA20'].head(365), self.SMA['SMA100'].head(365))
            shortPoints_100_5 = FindTheSellPoint(self.SMA['SMA20'].head(365), self.SMA['SMA100'].head(365))
        elif(indicator.lower() == 'ema'):
            print ("123")
        elif(indicator.lower() == 'rsi'):
            print ("123")
        elif(indicator.lower() == 'kd'):
            longPoints = FindTheBuyPoint(self.KD['K9'].head(365), self.KD['D9'].head(365))
            shortPoints = FindTheSellPoint(self.KD['K9'].head(365), self.KD['D9'].head(365))
        elif(indicator.lower() == 'macd'):
            print ("123")
        else:
            print ("Error: the content of argument is illegal")
            return
        return longPoints, shortPoints

In [6]:
#get the cryptocurrency price from DB

try:
    sql = "SELECT LAST, DATE, HIGH, LOW FROM btc_usd ORDER BY DATE"
    results = GetDataFromDB("140.118.126.136", "123", "1234567890", "test", sql)
    Dates = []
    ClosePrices = []
    HighPrices = []
    LowPrices = []
    for i in range(len(results)):
        Dates.append(results[i][1])
        ClosePrices.append(results[i][0])
        HighPrices.append(results[i][2])
        LowPrices.append(results[i][3])
        
    CoinPriceData = {
        'Date' : Dates,
        'Close' : ClosePrices,
        'High' : HighPrices,
        'Low' : LowPrices
    }

    CoinPriceData = pd.DataFrame(CoinPriceData)
except:
    print ("Error: unable to fetch data from DB")

In [84]:
sma5 = SMA(CoinPriceData['Close'], 5)
sma5 = sma5.fillna(0)
sma10 = SMA(CoinPriceData['Close'], 10)
sma10 = sma10.fillna(0)
sma20 = SMA(CoinPriceData['Close'], 20)
sma20 = sma20.fillna(0)
sma60 = SMA(CoinPriceData['Close'], 60)
sma60 = sma60.fillna(0)
sma120 = SMA(CoinPriceData['Close'], 120)
sma120 = sma120.fillna(0)
sma240 = SMA(CoinPriceData['Close'], 240)
sma240 = sma240.fillna(0)
rsi, up, down = RSI(CoinPriceData['Close'], 14)
rsi = rsi.fillna(0)
up = up.fillna(0)
down = down.fillna(0)
kdresult = KD(CoinPriceData)
k = kdresult['K9']
d = kdresult['D9']
macd = MACD(CoinPriceData, [12, 26, 9])
dif = macd['dif']
dem = macd['dem']
sql = "INSERT INTO indicators (DATE, SMA240, SMA120, SMA60, SMA20, SMA10, SMA5, RSI, K, D, DIF, DEM, UP, DOWN) VALUES "
for i in range(len(CoinPriceData['Date'])):
    if i != 0:
        sql += ","
    sql += "("
    sql += "'" + str(CoinPriceData['Date'][i]) + "'" + ","
    sql += str(sma240[i]) + ","
    sql += str(sma120[i]) + ","
    sql += str(sma60[i]) + ","
    sql += str(sma20[i]) + ","
    sql += str(sma10[i]) + ","
    sql += str(sma5[i]) + ","
    sql += str(rsi[i]) + ","
    sql += str(k[i]) + ","
    sql += str(d[i]) + ","
    sql += str(dif[i]) + ","
    sql += str(dem[i]) + ","
    sql += str(up[i]) + ","
    sql += str(down[i])
    sql += ")"
sql += ";"

In [85]:
print(sql)
db = pymysql.connect("140.118.126.136", "123", "1234567890", "test")
cursor = db.cursor()
cursor.execute(sql)
db.close()

INSERT INTO indicators (DATE, SMA240, SMA120, SMA60, SMA20, SMA10, SMA5, RSI, K, D, DIF, DEM, UP, DOWN) VALUES ('2014-04-15 00:00:00',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0),('2014-04-16 00:00:00',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0),('2014-04-17 00:00:00',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0),('2014-04-18 00:00:00',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0),('2014-04-19 00:00:00',0.0,0.0,0.0,0.0,0.0,508.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0),('2014-04-20 00:00:00',0.0,0.0,0.0,0.0,0.0,507.538,0.0,0.0,0.0,0.0,0.0,0.0,0.0),('2014-04-21 00:00:00',0.0,0.0,0.0,0.0,0.0,499.338,0.0,0.0,0.0,0.0,0.0,0.0,0.0),('2014-04-22 00:00:00',0.0,0.0,0.0,0.0,0.0,495.97799999999995,0.0,0.0,0.0,0.0,0.0,0.0,0.0),('2014-04-23 00:00:00',0.0,0.0,0.0,0.0,0.0,497.434,0.0,0.13344,0.04448,0.0,0.0,0.0,0.0),('2014-04-24 00:00:00',0.0,0.0,0.0,0.0,501.89,495.53000000000003,0.0,0.19898,0.09598,0.0,0.0,0.0,0.0),('2014-04-25 00:00:00',0.0,0.0,0.0,0.0,497.76899999999995,488.0,0.0,0

In [87]:
a = GetDataFromDB("140.118.126.136", "123", "1234567890", "test", "SELECT * FROM test.indicators")
print(a)

(('2014-04-15 00:00:00', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), ('2014-04-16 00:00:00', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), ('2014-04-17 00:00:00', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), ('2014-04-18 00:00:00', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), ('2014-04-19 00:00:00', 0.0, 0.0, 0.0, 0.0, 0.0, 508.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), ('2014-04-20 00:00:00', 0.0, 0.0, 0.0, 0.0, 0.0, 507.538, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), ('2014-04-21 00:00:00', 0.0, 0.0, 0.0, 0.0, 0.0, 499.338, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), ('2014-04-22 00:00:00', 0.0, 0.0, 0.0, 0.0, 0.0, 495.978, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), ('2014-04-23 00:00:00', 0.0, 0.0, 0.0, 0.0, 0.0, 497.434, 0.0, 0.13344, 0.04448, 0.0, 0.0, 0.0, 0.0), ('2014-04-24 00:00:00', 0.0, 0.0, 0.0, 0.0, 501.89, 495.53, 0.0, 0.19898, 0.09598, 0.0, 0.0, 0.0, 0.0), ('2014-04-25 00:00:00', 0.0, 0.0, 0.0, 0.0, 497.769, 488.0

In [7]:
rsiR('2018/03/12')
kdR('2018/03/12')

UPDATE INDICATORS SET RSI = 0.0 WHERE DATE = 2018/03/12


IndexError: tuple index out of range

In [18]:
#define a class variable with class Indicators
indicators = Indicators(CoinPriceData)

In [19]:
# get long, short points by all indicators
##LPoint_sma, SPoint_sma = indicators.getLong_ShortPoints('sma')
LPoint_sma, SPoint_ema = indicators.getLong_ShortPoints('ema')
LPoint_sma, SPoint_kd = indicators.getLong_ShortPoints('kd')
LPoint_sma, SPoint_rsi = indicators.getLong_ShortPoints('rsi')
LPoint_sma, SPoint_macd = indicators.getLong_ShortPoints('macd')

123
123
123


In [20]:
# sma
smaResult = indicators.getSMA()
smaResult

SMA5     SMA10      SMA20        SMA60       SMA120       SMA240
0          NaN       NaN        NaN          NaN          NaN          NaN
1          NaN       NaN        NaN          NaN          NaN          NaN
2          NaN       NaN        NaN          NaN          NaN          NaN
3          NaN       NaN        NaN          NaN          NaN          NaN
4      508.250       NaN        NaN          NaN          NaN          NaN
5      507.538       NaN        NaN          NaN          NaN          NaN
6      499.338       NaN        NaN          NaN          NaN          NaN
7      495.978       NaN        NaN          NaN          NaN          NaN
8      497.434       NaN        NaN          NaN          NaN          NaN
9      495.530   501.890        NaN          NaN          NaN          NaN
10     488.000   497.769        NaN          NaN          NaN          NaN
11     480.120   489.729        NaN          NaN          NaN          NaN
12     470.880   483.429        NaN          NaN          NaN          NaN
13     461.954   479.694        NaN          NaN          NaN          NaN
14     451.466   473.498        NaN          NaN          NaN          NaN
15     448.582   468.291        NaN          NaN          NaN          NaN
16     449.082   464.601        NaN          NaN          NaN          NaN
17     450.052   460.466        NaN          NaN          NaN          NaN
18     449.232   455.593        NaN          NaN          NaN          NaN
19     447.404   449.435   475.6625          NaN          NaN          NaN
20     443.680   446.131   471.9500          NaN          NaN          NaN
21     437.840   443.461   466.5950          NaN          NaN          NaN
22     437.070   443.561   463.4950          NaN          NaN          NaN
23     437.210   443.221   461.4575          NaN          NaN          NaN
24     440.930   444.167   458.8325          NaN          NaN          NaN
25     442.150   442.915   455.6030          NaN          NaN          NaN
26     443.852   440.846   452.7235          NaN          NaN          NaN
27     442.216   439.643   450.0545          NaN          NaN          NaN
28     443.216   440.213   447.9030          NaN          NaN          NaN
29     441.610   441.270   445.3525          NaN          NaN          NaN
...        ...       ...        ...          ...          ...          ...
1472  7630.436  7592.808  7565.8690  8114.014500  8710.382833  9375.154500
1473  7445.856  7518.608  7482.2090  8109.897833  8672.667000  9385.642000
1474  7269.056  7444.248  7421.4140  8109.269500  8645.095333  9396.499500
1475  7044.696  7335.268  7368.8890  8100.966167  8614.557000  9405.511583
1476  6780.040  7197.918  7306.9590  8083.135667  8589.984500  9413.500333
1477  6569.300  7099.868  7265.0840  8079.602333  8570.640333  9422.480333
1478  6526.300  6986.078  7222.8190  8074.637333  8548.324500  9431.530750
1479  6514.620  6891.838  7192.1240  8076.544000  8535.499500  9441.897000
1480  6554.320  6799.508  7173.8740  8077.002333  8534.182000  9451.756167
1481  6634.460  6707.250  7138.4240  8075.997333  8525.788667  9461.643667
1482  6675.120  6622.210  7107.5090  8057.807333  8517.977000  9471.088250
1483  6590.280  6558.290  7038.4490  8024.385667  8499.791167  9477.049917
1484  6485.100  6499.860  6972.0540  7988.572333  8478.825333  9483.016167
1485  6371.340  6462.830  6899.0490  7957.707333  8458.725333  9488.708667
1486  6274.340  6454.400  6826.1590  7930.209000  8443.483667  9494.646167
1487  6159.760  6417.440  6758.6540  7896.894000  8420.770333  9498.175750
1488  6176.080  6383.180  6684.6290  7861.380667  8400.797833  9500.239917
1489  6123.618  6304.359  6598.0985  7811.892167  8371.237750  9500.956125
1490  6138.298  6254.819  6527.1635  7769.015500  8338.741917  9503.090708
1491  6157.838  6216.089  6461.6695  7727.430500  8306.764417  9505.685292
1492  6198.678  6179.219  6400.7145  7684.668833  8268.005250  9509.055708
1493  6296.878  6236.479  6397.3845

In [10]:
# Draw SMA curve
totalDate = 365
plt.figure(figsize = (16, 8))

x = [i for i in range(len(CoinPriceData.head(totalDate)))]
#x = list(dfs['Date'].head(totalDate))

## draw sma5
sma5c = [smaResult['SMA5'][i] for i in range(len(smaResult['SMA5'].head(totalDate)))]
plt.plot(x, sma5c, 'r', label = 'sma_5 days')

## draw sma10
sma5c = [smaResult['SMA10'][i] for i in range(len(smaResult['SMA10'].head(totalDate)))]
plt.plot(x, sma5c, 'g', label = 'sma_10 days')

## draw sma20
sma20c = [smaResult['SMA20'][i] for i in range(len(smaResult['SMA20'].head(totalDate)))]
plt.plot(x, sma20c, 'b', label = 'sma_20 days')

## draw sma60
sma5c = [smaResult['SMA60'][i] for i in range(len(smaResult['SMA60'].head(totalDate)))]
plt.plot(x, sma5c, 'c', label = 'sma_60 days')

## draw sma120
sma100c = [smaResult['SMA120'][i] for i in range(len(smaResult['SMA120'].head(totalDate)))]
plt.plot(x, sma100c, 'm', label = 'sma_120 days')

## draw sma240
sma5c = [smaResult['SMA240'][i] for i in range(len(smaResult['SMA240'].head(totalDate)))]
plt.plot(x, sma5c, 'y', label = 'sma_240 days')

## draw all close prices
plt.plot(x, CoinPriceData['Close'].head(totalDate), label = 'Close price', color = 'k')

## draw the intersection points of three curves
# FindTheIntersection(sma5c, sma20c)

#p#icture setting
plt.xlabel("Index")
plt.ylabel("Close Prices")
plt.title("Technical Analysis - Simple Moving Average")
plt.legend(loc = 'upper right')

##show picture
plt.show()

NameError: name 'smaResult' is not defined

<Figure size 1152x576 with 0 Axes>

In [11]:
# ema
emaResult = indicators.getEMA()

NameError: name 'indicators' is not defined

In [12]:
# Draw EMA curve
totalDate = 365
plt.figure(figsize = (16, 8))

x = [i for i in range(len(emaResult.head(totalDate)))]
#x = list(dfs['Date'].head(totalDate))

##draw ema5
ema5c = [emaResult['EMA5'][i] for i in range(len(emaResult['EMA5'].head(totalDate)))]
plt.plot(x, ema5c, 'r', label = 'ema_5 days')

##draw ema20
ema20c = [emaResult['EMA20'][i] for i in range(len(emaResult['EMA20'].head(totalDate)))]
plt.plot(x, ema20c, 'g', label = 'ema_20 days')

##draw ema20
ema100c = [emaResult['EMA100'][i] for i in range(len(emaResult['EMA100'].head(totalDate)))]
plt.plot(x, ema100c, 'b', label = 'ema_100 days')

##draw all close prices
plt.plot(x, CoinPriceData['Close'].head(totalDate), label = 'Close price', color = 'k')

#p#icture setting
plt.xlabel("Index")
plt.ylabel("Close Prices")
plt.title("Technical Analysis - Exponential Moving Average")
plt.legend(loc = 'upper right')

##show picture
plt.show()

NameError: name 'emaResult' is not defined

<Figure size 1152x576 with 0 Axes>

In [13]:
# KD
#stockData_1101 = pd.read_csv(r"C:\Users\Mike\Desktop\1101.TW.csv", converters={"Open":float, "High":float, "Low":float, "Close":float})
#result = pd.DataFrame(stockData_1101)
#data = KD(result)
#dfs_KD = pd.DataFrame(data)

kdResult = indicators.getKD()

NameError: name 'indicators' is not defined

In [14]:
# Draw the KD Curve
totalDate = 365
plt.figure(figsize = (16, 8))

x = [i for i in range(len(CoinPriceData['Close'].head(totalDate)))]
#x = list(dfs['Date'].head(totalDate))

## Draw K curve
Kc = [kdResult['K9'][i] for i in range(len(kdResult['K9'].head(totalDate)))]
plt.plot(x, Kc, 'b', label = 'K curve')

## Draw D curve
Dc = [kdResult['D9'][i] for i in range(len(kdResult['D9'].head(totalDate)))]
plt.plot(x, Dc, 'y', label = 'D curve')

##draw all close prices
#plt.plot(x, CoinPriceData['Close'].head(totalDate), label = 'Close price', color = 'k')

##draw all intersection point of two curves
longPoints, shortPoints = indicators.getLong_ShortPoints("KD")

longc = [kdResult['D9'][longPoints[longPoints <= 365][i]] for i in range(len(longPoints[longPoints <= 365]))]
plt.plot(longPoints[longPoints <= 365], longc, 'go', label = 'long points')

shortc = [kdResult['D9'][shortPoints[shortPoints <= 365][i]] for i in range(len(shortPoints[shortPoints <= 365]))]
plt.plot(shortPoints[shortPoints <= 365], shortc, 'ro', label = 'short points')

#p#icture setting
plt.xlabel("Index")
plt.ylabel("Percentages")
plt.title("Technical Analysis - Stochastic Oscillator, KD")
plt.legend(loc = 'upper right')

##show picture
plt.show()

NameError: name 'kdResult' is not defined

<Figure size 1152x576 with 0 Axes>

In [15]:
# RSI
#stockData_1101 = pd.read_csv(r"C:\Users\Mike\Desktop\1101.TW.csv", converters={"Open":float, "High":float, "Low":float, "Close":float})
#result = pd.DataFrame(stockData_1101)

period = 14

#rsi = RSI(result['Close'], period)


In [16]:
# Draw RSI curve
totalDate = 365
plt.figure(figsize = (16, 8))

x = [i for i in range(len(dfs_close.tail(totalDate)))]
#x = list(dfs['Date'].head(totalDate))

## Draw rsi curve
rsic = plt.plot(x, rsi.tail(totalDate), 'k', label = 'RSI_curve')

#p#icture setting
plt.xlabel("Index")
plt.ylabel("Percentages")
plt.title("Technical Analysis - Relative Strength Index, RSI")
plt.legend(loc = 'upper right')

##show picture
plt.show()

NameError: name 'dfs_close' is not defined

<Figure size 1152x576 with 0 Axes>

In [17]:
# MACD
stockData_1101 = pd.read_csv(r"C:\Users\Mike\Desktop\1101.TW.csv", converters={"Open":float, "High":float, "Low":float, "Close":float})
result = pd.DataFrame(stockData_1101)

dif, dem = MACD(result, [12,26,9])
# 小數點下兩位正確

FileNotFoundError: File b'C:\\Users\\Mike\\Desktop\\1101.TW.csv' does not exist

In [ ]:
# draw MACD Curve
totalDate = 365
plt.figure(figsize = (16, 8))

x = [i for i in range(len(dfs_close.head(totalDate)))]
#x = list(dfs['Date'].head(totalDate))

## Draw DIF curve
difc = plt.plot(x, dif.head(totalDate), 'k', label = 'DIF_curve')

## Draw DEM curve
demc = plt.plot(x, dem.head(totalDate), 'b', label = 'DEM_curve')

## Draw the intersection points of two curves
buyPoint = FindTheBuyPoint(dif.head(totalDate), dem.head(totalDate))
sellPoint = FindTheSellPoint(dif.head(totalDate), dem.head(totalDate))
buyPoints = [buyPoint[i] for i in range(len(buyPoint))]
sellPoints = [sellPoint[i] for i in range(len(sellPoint))]

plt.plot(buyPoints, dem[buyPoints], "ro", label = "Buy Points")
plt.plot(sellPoints, dem[sellPoints], "go", label = "Sell Points")

#picture setting
plt.xlabel("Index")
plt.ylabel("Close Price")
plt.title("Technical Analysis - Moving Average Convergence / Divergence, MACD")
plt.legend(loc = 'upper right')

##show picture
plt.show()